In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1ot40UkYIUGOKX6cO28mOsGHwC-srClZB", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 Sequence & Expert Parallelism: The Final Two Dimensions

*Part 5 of the Vizuara series on 5D Parallelism from Scratch*
*Estimated time: 50 minutes*

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/5d-parallelism-from-scratch/practice/5/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 1. Why Does This Matter?

We have now built three of the five dimensions of modern parallelism: **Data Parallelism** (split the batch), **Tensor Parallelism** (split the layers), and **Pipeline Parallelism** (split the depth). These three alone powered most large model training through 2022.

But two critical challenges remain:

**Challenge 1 — Sequences are getting absurdly long.** GPT-3 was trained with 2,048 tokens of context. Claude now handles 200,000. Google Gemini processes over 1,000,000. The memory required for attention scales as $O(n^2)$ — at 128K tokens, a single attention matrix consumes over 60 GB. No single GPU can hold that. We need **Sequence Parallelism**.

**Challenge 2 — We want massive models but cannot afford to activate all parameters.** DeepSeek-V3 has 671 billion parameters, yet only 37 billion are active for any given token. The trick? A **Mixture of Experts** — hundreds of specialist sub-networks, with a learned router that dispatches each token to just a few relevant experts. We need **Expert Parallelism** to place those experts across GPUs.

By the end of this notebook, you will have:
- Built a **Ring Attention** simulation that distributes a long sequence across 4 GPUs in a ring, with K and V blocks rotating until every query has seen every key
- Implemented a complete **Mixture-of-Experts layer** with top-K gating, token routing, and load-balancing loss
- Verified both against their non-parallel counterparts
- Understood every equation that makes these systems work

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_setup_imports.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup — Run this cell first
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import math
from typing import Tuple, List

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Sequence Parallelism Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_sequence_parallelism_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


---

# Part A: Sequence Parallelism

## 2. Building Intuition — Sequence Parallelism

Imagine you are the editor of a publishing house, and you need to translate a 500-page book into another language. You have four translators on staff. The obvious approach: give each translator 125 pages.

This works beautifully for most of the text. But occasionally a pronoun on page 126 refers back to a character introduced on page 40 — now translator 2 needs information from translator 1. The translators must coordinate at these boundary points.

This is exactly the situation in Transformer models processing long sequences:
- **The "pages"** are tokens in the sequence
- **The "translators"** are GPUs
- **The "cross-references"** are attention — every token must attend to every other token

There are two flavors of sequence parallelism, each solving a different problem:

### Flavor 1: Megatron-Style SP — Fixing Waste in Tensor Parallelism

Recall from Notebook 3 that Tensor Parallelism (TP) splits the weight matrices across GPUs. The attention and MLP computations are split. But what about **LayerNorm** and **Dropout**? In standard TP, these operations are simply **replicated** on every GPU — each GPU computes the identical LayerNorm on the identical input. That is pure waste.

Megatron-Style SP fixes this: instead of replicating LayerNorm and Dropout, we **split them along the sequence dimension**. Each GPU handles `seq_len/N` tokens for these operations. The communication pattern changes from AllReduce to ReduceScatter + AllGather, but the total bytes transferred stay the same. We get a memory savings for free.

### Flavor 2: Ring Attention — For Truly Long Sequences

When sequences reach 100K to 1M+ tokens, even the attention computation itself is too large for a single GPU. Ring Attention solves this by arranging GPUs in a logical ring:

1. Split the sequence into chunks — each GPU holds one chunk of Q, K, and V
2. Each GPU computes attention between its local Q and the K, V it currently holds
3. Pass K and V to the next GPU in the ring
4. Repeat until every Q chunk has seen every K, V chunk
5. Combine the partial attention results using a numerically stable trick

The beautiful part: while one GPU is computing attention, the next GPU's K, V is already being transmitted. **Communication overlaps with computation.**

We will focus our implementation on Ring Attention, since it is the more general and conceptually interesting of the two.

### 🤔 Think About This

Before we proceed, consider:
- In Ring Attention, each GPU computes partial attention scores across multiple rounds. But softmax requires the *global* maximum for numerical stability. How can each GPU compute a correct softmax if it only sees a fraction of the keys at a time?
- Why arrange the GPUs in a *ring* rather than having all GPUs broadcast their K, V to everyone?

*Hold these questions in mind. The answers will emerge from the math below.*

In [ ]:
# Quick status check
print("Section checkpoint ✓")

In [ ]:
#@title 🎧 Listen: Ring Attention Math
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_ring_attention_math.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics — Ring Attention

### Standard Attention (the baseline)

For a single head, standard scaled dot-product attention on a full sequence is:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V$$

where $Q, K, V \in \mathbb{R}^{n \times d_k}$ and $n$ is the sequence length. The attention matrix $QK^T$ is $n \times n$, which is where the $O(n^2)$ memory comes from.

**Computationally:** We compute an $n \times n$ score matrix, apply softmax row-wise, then multiply by $V$ to get the output. At $n = 128{,}000$ and $d_k = 128$, the score matrix alone is $128{,}000^2 \times 4$ bytes $\approx$ 62 GB in fp32. That does not fit on any single GPU.

### Chunked Attention

Suppose we split the sequence into $P$ chunks of size $c = n / P$. GPU $i$ holds:
- $Q_i \in \mathbb{R}^{c \times d_k}$ — its query chunk
- $K_i, V_i \in \mathbb{R}^{c \times d_k}$ — its key and value chunks

The full attention output for GPU $i$'s queries requires computing against *all* keys and values:

$$O_i = \text{softmax}\left(\frac{Q_i [K_0; K_1; \ldots; K_{P-1}]^T}{\sqrt{d_k}}\right) [V_0; V_1; \ldots; V_{P-1}]$$

**Let us plug in some simple numbers.** With $P = 4$ GPUs and $n = 1024$ tokens, each chunk is $c = 256$ tokens. GPU 0 needs to compute attention of its 256 queries against all 1,024 keys. Instead of materializing the full $256 \times 1024$ score matrix at once, we compute four blocks of $256 \times 256$ scores — one per round — and combine them.

### The Online Softmax Trick

The key mathematical insight is that we can compute softmax **incrementally**. For GPU $i$, after processing the key chunk from round $j$, we have partial scores:

$$S_{ij} = \frac{Q_i K_j^T}{\sqrt{d_k}} \in \mathbb{R}^{c \times c}$$

We maintain running statistics for each query row $q$:
- $m_q$: the running maximum score (for numerical stability)
- $\ell_q$: the running sum of exponentiated scores
- $o_q$: the running weighted output

After seeing a new chunk of scores $s_{\text{new}}$ (a row of $S_{ij}$):

$$m_{\text{new}} = \max(m_q, \max(s_{\text{new}}))$$
$$\ell_q \leftarrow \ell_q \cdot e^{m_q - m_{\text{new}}} + \sum_k e^{s_{\text{new},k} - m_{\text{new}}}$$
$$o_q \leftarrow o_q \cdot e^{m_q - m_{\text{new}}} + \sum_k e^{s_{\text{new},k} - m_{\text{new}}} \cdot V_j[k]$$

After all $P$ rounds:

$$\text{output}_q = o_q / \ell_q$$

**What this says computationally:** We process one block of keys at a time, updating the running maximum, running sum of exponentials, and running weighted sum. The $e^{m_{\text{old}} - m_{\text{new}}}$ correction rescales the old statistics when a new maximum is discovered. After all chunks, we divide by the total sum to get the correct softmax-weighted output — identical to computing standard attention on the full sequence.

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_standard_attention_baseline.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let Us Build It — Ring Attention

### 4.1 Standard Attention (Baseline)

First, let us implement standard attention so we have a ground truth to verify against.

In [ ]:
def standard_attention(Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor) -> torch.Tensor:
    """
    Standard scaled dot-product attention.

    Args:
        Q: (seq_len, d_k)
        K: (seq_len, d_k)
        V: (seq_len, d_k)

    Returns:
        output: (seq_len, d_k)
    """
    d_k = Q.size(-1)
    # Score matrix: (seq_len, seq_len)
    scores = Q @ K.T / math.sqrt(d_k)
    # Softmax over keys dimension
    attn_weights = F.softmax(scores, dim=-1)
    # Weighted sum of values
    output = attn_weights @ V
    return output

Let us run a quick test to verify our baseline attention implementation works correctly.

In [ ]:
# Quick test
seq_len, d_k = 16, 8
Q_test = torch.randn(seq_len, d_k)
K_test = torch.randn(seq_len, d_k)
V_test = torch.randn(seq_len, d_k)

output_standard = standard_attention(Q_test, K_test, V_test)
print(f"Input shape:  Q={Q_test.shape}, K={K_test.shape}, V={V_test.shape}")
print(f"Output shape: {output_standard.shape}")
print(f"Output[:2, :4]:\n{output_standard[:2, :4]}")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_chunking_sequence.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Chunking the Sequence

Now let us split Q, K, V into chunks and assign each to a simulated "GPU."

In [ ]:
def chunk_sequence(Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor,
                   num_gpus: int) -> Tuple[List[torch.Tensor], ...]:
    """
    Split Q, K, V evenly across simulated GPUs.

    Args:
        Q, K, V: (seq_len, d_k)
        num_gpus: number of chunks

    Returns:
        Q_chunks, K_chunks, V_chunks: lists of (chunk_size, d_k) tensors
    """
    Q_chunks = list(Q.chunk(num_gpus, dim=0))
    K_chunks = list(K.chunk(num_gpus, dim=0))
    V_chunks = list(V.chunk(num_gpus, dim=0))
    return Q_chunks, K_chunks, V_chunks

Now we create a longer sequence, chunk it across 4 simulated GPUs, and inspect the resulting shapes.

In [ ]:
# Create a longer sequence and chunk it
NUM_GPUS = 4
seq_len, d_k = 1024, 64
torch.manual_seed(42)

Q_full = torch.randn(seq_len, d_k)
K_full = torch.randn(seq_len, d_k)
V_full = torch.randn(seq_len, d_k)

Q_chunks, K_chunks, V_chunks = chunk_sequence(Q_full, K_full, V_full, NUM_GPUS)

chunk_size = seq_len // NUM_GPUS
print(f"Full sequence length: {seq_len}")
print(f"Number of GPUs: {NUM_GPUS}")
print(f"Chunk size per GPU: {chunk_size}")
for i in range(NUM_GPUS):
    print(f"  GPU {i}: Q={Q_chunks[i].shape}, K={K_chunks[i].shape}, V={V_chunks[i].shape}")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_ring_attention_step.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Ring Attention — Step by Step

Here is the core algorithm. Each GPU starts with its own K, V chunk. In each round, every GPU:
1. Computes partial attention scores between its Q and the K chunk it currently holds
2. Updates running statistics (online softmax)
3. Passes its K, V chunk to the next GPU in the ring

The function below implements a single step — processing one K,V block and updating the running online softmax statistics.

In [ ]:
def ring_attention_step(Q_local: torch.Tensor, K_block: torch.Tensor,
                        V_block: torch.Tensor, running_max: torch.Tensor,
                        running_sum: torch.Tensor, running_output: torch.Tensor,
                        d_k: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    One step of ring attention: process one K,V block and update running statistics.

    Uses the online softmax trick for numerical stability.

    Args:
        Q_local: (chunk_size, d_k) — this GPU's queries (fixed)
        K_block: (chunk_size, d_k) — the K chunk currently held
        V_block: (chunk_size, d_k) — the V chunk currently held
        running_max: (chunk_size, 1) — running row-wise max of scores
        running_sum: (chunk_size, 1) — running sum of exp(scores - max)
        running_output: (chunk_size, d_k) — running weighted output
        d_k: dimension for scaling

    Returns:
        Updated (running_max, running_sum, running_output)
    """
    # Compute attention scores for this block: (chunk_size, chunk_size)
    scores = Q_local @ K_block.T / math.sqrt(d_k)

    # Row-wise max of the new scores: (chunk_size, 1)
    block_max = scores.max(dim=-1, keepdim=True).values

The key insight of online softmax: we update the running maximum and rescale previous statistics. This lets us combine partial attention results from different K,V blocks without ever materializing the full attention matrix.

In [ ]:
    # (Continued from ring_attention_step)
    # New global max across old and new
    new_max = torch.maximum(running_max, block_max)

    # Rescale old statistics to the new maximum
    correction_old = torch.exp(running_max - new_max)
    correction_new = torch.exp(scores - new_max)  # (chunk_size, chunk_size)

    # Update running sum of exponentials
    new_sum = running_sum * correction_old + correction_new.sum(dim=-1, keepdim=True)

    # Update running weighted output
    new_output = running_output * correction_old + correction_new @ V_block

    return new_max, new_sum, new_output

**Important:** The code above shows the logic in two parts for readability, but the actual `ring_attention_step` function is defined as a single unit. Let us now define the complete function for use in the simulation.

In [ ]:
def ring_attention_step(Q_local, K_block, V_block,
                        running_max, running_sum, running_output, d_k):
    """One step of ring attention with online softmax update."""
    scores = Q_local @ K_block.T / math.sqrt(d_k)
    block_max = scores.max(dim=-1, keepdim=True).values
    new_max = torch.maximum(running_max, block_max)
    correction_old = torch.exp(running_max - new_max)
    correction_new = torch.exp(scores - new_max)
    new_sum = running_sum * correction_old + correction_new.sum(dim=-1, keepdim=True)
    new_output = running_output * correction_old + correction_new @ V_block
    return new_max, new_sum, new_output

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_full_ring_simulation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now let us simulate the full ring. We will track what happens on each GPU across all rounds. Each GPU initializes running statistics, then iterates through `num_gpus` rounds — computing attention with whatever K,V block it currently holds and rotating K,V to the next GPU after each round.

In [ ]:
def ring_attention(Q_chunks: List[torch.Tensor], K_chunks: List[torch.Tensor],
                   V_chunks: List[torch.Tensor]) -> torch.Tensor:
    """
    Full ring attention across all simulated GPUs.

    Args:
        Q_chunks: list of (chunk_size, d_k) query chunks
        K_chunks: list of (chunk_size, d_k) key chunks
        V_chunks: list of (chunk_size, d_k) value chunks

    Returns:
        output: (seq_len, d_k) — concatenated outputs from all GPUs
    """
    num_gpus = len(Q_chunks)
    chunk_size, d_k = Q_chunks[0].shape

    # Each GPU maintains running statistics
    running_maxes = [torch.full((chunk_size, 1), -float('inf'))
                     for _ in range(num_gpus)]
    running_sums = [torch.zeros(chunk_size, 1) for _ in range(num_gpus)]
    running_outputs = [torch.zeros(chunk_size, d_k) for _ in range(num_gpus)]

    # Each GPU starts holding its own K, V chunk
    current_kv = [(K_chunks[i].clone(), V_chunks[i].clone())
                  for i in range(num_gpus)]

The inner loop: for each round, every GPU computes attention with its current K,V block, then K,V blocks rotate one step clockwise around the ring.

In [ ]:
    # (Continued: ring rounds and final normalization)
    for round_idx in range(num_gpus):
        # Each GPU computes attention with whatever K,V it currently holds
        for gpu_id in range(num_gpus):
            K_block, V_block = current_kv[gpu_id]
            running_maxes[gpu_id], running_sums[gpu_id], running_outputs[gpu_id] = \
                ring_attention_step(
                    Q_chunks[gpu_id], K_block, V_block,
                    running_maxes[gpu_id], running_sums[gpu_id],
                    running_outputs[gpu_id], d_k
                )

        # Rotate K,V: each GPU sends its K,V to the next GPU in the ring
        if round_idx < num_gpus - 1:
            new_kv = [None] * num_gpus
            for gpu_id in range(num_gpus):
                next_gpu = (gpu_id + 1) % num_gpus
                new_kv[next_gpu] = current_kv[gpu_id]
            current_kv = new_kv

    # Final normalization: output = running_output / running_sum
    outputs = []
    for gpu_id in range(num_gpus):
        outputs.append(running_outputs[gpu_id] / running_sums[gpu_id])

    return torch.cat(outputs, dim=0)

Again, for the simulation to work as a single callable function, here is the complete `ring_attention` assembled:

In [ ]:
def ring_attention(Q_chunks, K_chunks, V_chunks):
    """Full ring attention across all simulated GPUs."""
    num_gpus = len(Q_chunks)
    chunk_size, d_k = Q_chunks[0].shape
    running_maxes = [torch.full((chunk_size, 1), -float('inf'))
                     for _ in range(num_gpus)]
    running_sums = [torch.zeros(chunk_size, 1) for _ in range(num_gpus)]
    running_outputs = [torch.zeros(chunk_size, d_k) for _ in range(num_gpus)]
    current_kv = [(K_chunks[i].clone(), V_chunks[i].clone())
                  for i in range(num_gpus)]
    for round_idx in range(num_gpus):
        for gpu_id in range(num_gpus):
            K_block, V_block = current_kv[gpu_id]
            running_maxes[gpu_id], running_sums[gpu_id], \
                running_outputs[gpu_id] = ring_attention_step(
                    Q_chunks[gpu_id], K_block, V_block,
                    running_maxes[gpu_id], running_sums[gpu_id],
                    running_outputs[gpu_id], d_k)
        if round_idx < num_gpus - 1:
            new_kv = [None] * num_gpus
            for gpu_id in range(num_gpus):
                new_kv[(gpu_id + 1) % num_gpus] = current_kv[gpu_id]
            current_kv = new_kv
    outputs = [running_outputs[i] / running_sums[i] for i in range(num_gpus)]
    return torch.cat(outputs, dim=0)

# Run ring attention
output_ring = ring_attention(Q_chunks, K_chunks, V_chunks)
print(f"Ring attention output shape: {output_ring.shape}")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_verification.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 Verifying Correctness

The moment of truth: does our ring attention produce the same result as standard attention?

In [ ]:
# ✅ Verification: Ring Attention vs Standard Attention
output_standard_full = standard_attention(Q_full, K_full, V_full)

max_error = (output_ring - output_standard_full).abs().max().item()
mean_error = (output_ring - output_standard_full).abs().mean().item()

print("=" * 50)
print("Ring Attention Correctness Check")
print("=" * 50)
print(f"Max absolute error:  {max_error:.2e}")
print(f"Mean absolute error: {mean_error:.2e}")
print(f"Tolerance:           1e-5")
print()

if max_error < 1e-5:
    print("✅ Ring Attention matches standard attention perfectly!")
    print("   The online softmax trick produces numerically identical results.")
else:
    print("⚠️ Errors detected — check the implementation.")

# Show a few output values for comparison
print(f"\nFirst 4 values, first 4 dims:")
print(f"  Standard: {output_standard_full[:4, :4].tolist()}")
print(f"  Ring:     {output_ring[:4, :4].tolist()}")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_viz_ring_rotation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization 1: The Ring Rotation

Let us visualize how K and V blocks rotate around the ring across rounds. This is the core communication pattern of Ring Attention. We draw the 4 GPUs in a circle for each round, color-coded by which K,V block they currently hold.

In [ ]:
fig, axes = plt.subplots(1, NUM_GPUS, figsize=(16, 5))
fig.suptitle("Ring Attention: K,V Block Rotation Across Rounds",
             fontsize=14, fontweight='bold', y=1.02)

colors = ['#2196F3', '#4CAF50', '#FF9800', '#E91E63']
gpu_labels = [f'GPU {i}' for i in range(NUM_GPUS)]

for round_idx in range(NUM_GPUS):
    ax = axes[round_idx]
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_title(f'Round {round_idx}', fontsize=12, fontweight='bold')
    ax.axis('off')

    # Draw 4 GPUs in a circle
    angles = [math.pi/2 - i * 2 * math.pi / NUM_GPUS
              for i in range(NUM_GPUS)]
    positions = [(math.cos(a), math.sin(a)) for a in angles]

    for gpu_id in range(NUM_GPUS):
        x, y = positions[gpu_id]
        kv_origin = (gpu_id - round_idx) % NUM_GPUS
        circle = plt.Circle((x, y), 0.35, color=colors[kv_origin],
                            alpha=0.8)
        ax.add_patch(circle)
        ax.text(x, y + 0.08, f'GPU {gpu_id}', ha='center',
                va='center', fontsize=8, fontweight='bold', color='white')
        ax.text(x, y - 0.12, f'K,V from\nGPU {kv_origin}',
                ha='center', va='center', fontsize=6, color='white')

Now we add ring arrows showing the K,V transfer direction, and a legend for the color coding.

In [ ]:
    # Draw ring arrows (except last round)
    if round_idx < NUM_GPUS - 1:
        for gpu_id in range(NUM_GPUS):
            next_gpu = (gpu_id + 1) % NUM_GPUS
            x1, y1 = positions[gpu_id]
            x2, y2 = positions[next_gpu]
            dx, dy = x2 - x1, y2 - y1
            length = math.sqrt(dx**2 + dy**2)
            # Shorten arrow to not overlap circles
            shrink = 0.4 / length
            ax.annotate('', xy=(x2 - dx*shrink, y2 - dy*shrink),
                       xytext=(x1 + dx*shrink, y1 + dy*shrink),
                       arrowprops=dict(arrowstyle='->', color='gray',
                                      lw=1.5, alpha=0.5))

# Legend
legend_patches = [mpatches.Patch(color=colors[i],
                  label=f'K,V originally from GPU {i}')
                  for i in range(NUM_GPUS)]
fig.legend(handles=legend_patches, loc='lower center', ncol=NUM_GPUS,
           fontsize=9, bbox_to_anchor=(0.5, -0.05))

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_viz_attention_blocks.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization 2: Attention Matrix Filled Block-by-Block

Each round, a new block-column of the attention matrix is computed. Let us visualize this progressive filling. We compute the full score matrix and then show which blocks have been computed after each round.

In [ ]:
fig, axes = plt.subplots(1, NUM_GPUS + 1, figsize=(18, 4))
fig.suptitle("Attention Matrix Blocks Computed Per Round",
             fontsize=14, fontweight='bold', y=1.05)

# Compute the full attention score matrix for visualization
scores_full = Q_full @ K_full.T / math.sqrt(d_k)
scores_np = scores_full.detach().numpy()

# Progressively show which blocks are computed
for round_idx in range(NUM_GPUS):
    ax = axes[round_idx]
    # Create a masked version showing only blocks computed so far
    visible = np.full_like(scores_np, np.nan)
    for r in range(round_idx + 1):
        for gpu_id in range(NUM_GPUS):
            kv_origin = (gpu_id - r) % NUM_GPUS
            row_start = gpu_id * chunk_size
            row_end = (gpu_id + 1) * chunk_size
            col_start = kv_origin * chunk_size
            col_end = (kv_origin + 1) * chunk_size
            visible[row_start:row_end, col_start:col_end] = \
                scores_np[row_start:row_end, col_start:col_end]

    im = ax.imshow(visible, cmap='viridis', aspect='auto',
                   vmin=scores_np.min(), vmax=scores_np.max())
    ax.set_title(f'After Round {round_idx}', fontsize=10, fontweight='bold')
    ax.set_xlabel('Key positions')
    if round_idx == 0:
        ax.set_ylabel('Query positions')

We also add grid lines between chunks to make the block structure clear, and show the full attention matrix for comparison.

In [ ]:
    # Draw grid lines between chunks
    for i in range(1, NUM_GPUS):
        ax.axhline(y=i*chunk_size - 0.5, color='white', linewidth=1.5)
        ax.axvline(x=i*chunk_size - 0.5, color='white', linewidth=1.5)

# Final panel: full matrix
ax = axes[NUM_GPUS]
im = ax.imshow(scores_np, cmap='viridis', aspect='auto')
ax.set_title('Full Attention\n(standard)', fontsize=10, fontweight='bold')
ax.set_xlabel('Key positions')
for i in range(1, NUM_GPUS):
    ax.axhline(y=i*chunk_size - 0.5, color='white', linewidth=1.5)
    ax.axvline(x=i*chunk_size - 0.5, color='white', linewidth=1.5)

plt.colorbar(im, ax=axes.tolist(), shrink=0.8, label='Attention score')
plt.tight_layout()
plt.show()

print("Each round fills one diagonal band of blocks.")
print("After 4 rounds, the full attention matrix has been computed — "
      "but no single GPU ever held the entire matrix!")

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_todo1_rotation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. 🔧 Your Turn — Sequence Parallelism

### TODO 1: Implement the K,V Ring Rotation

In Ring Attention, after each round, every GPU sends its K and V chunk to the next GPU in the ring (clockwise). This is the fundamental communication primitive.

Implement the `rotate_kv_ring` function below.

In [ ]:
def rotate_kv_ring(kv_pairs: List[Tuple[torch.Tensor, torch.Tensor]]
                   ) -> List[Tuple[torch.Tensor, torch.Tensor]]:
    """
    Rotate K,V pairs one step clockwise around the ring.

    GPU 0's K,V goes to GPU 1, GPU 1's goes to GPU 2,
    ..., GPU (P-1)'s goes to GPU 0.

    Args:
        kv_pairs: list of (K, V) tuples, one per GPU.

    Returns:
        new_kv_pairs: list of (K, V) tuples after rotation.
    """
    num_gpus = len(kv_pairs)

    # ============ TODO ============
    # Implement one clockwise rotation of K,V pairs.
    # After rotation:
    #   - GPU 1 should hold what GPU 0 had
    #   - GPU 2 should hold what GPU 1 had
    #   - ...
    #   - GPU 0 should hold what GPU (P-1) had
    #
    # Hint: This is a circular shift.
    # ==============================

    new_kv_pairs = ???  # YOUR CODE HERE

    return new_kv_pairs

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_todo1_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us test the rotation with identifiable K,V pairs — each GPU's tensors are filled with its index so we can easily verify which data moved where.

In [ ]:
# ✅ Verification: Test your rotation function

# Create identifiable K,V pairs (filled with GPU index)
test_kv = [(torch.full((4, 4), float(i)),
            torch.full((4, 4), float(i) + 0.5))
           for i in range(4)]

print("Before rotation:")
for i, (k, v) in enumerate(test_kv):
    print(f"  GPU {i}: K[0,0]={k[0,0].item():.1f}, "
          f"V[0,0]={v[0,0].item():.1f}")

rotated = rotate_kv_ring(test_kv)

print("\nAfter one clockwise rotation:")
for i, (k, v) in enumerate(rotated):
    print(f"  GPU {i}: K[0,0]={k[0,0].item():.1f}, "
          f"V[0,0]={v[0,0].item():.1f}")

Now we verify every GPU received data from its predecessor in the ring.

In [ ]:
# Verify correctness
expected_sources = [3, 0, 1, 2]
all_correct = True
for i, (k, v) in enumerate(rotated):
    expected_k = float(expected_sources[i])
    expected_v = float(expected_sources[i]) + 0.5
    if k[0, 0].item() != expected_k or v[0, 0].item() != expected_v:
        print(f"  ❌ GPU {i}: expected K from GPU {expected_sources[i]}, "
              f"got K[0,0]={k[0,0].item():.1f}")
        all_correct = False

if all_correct:
    print("\n✅ Rotation is correct! Each GPU received K,V from its predecessor.")
else:
    print("\n⚠️ Check your rotation logic.")

In [ ]:
#@title 🎧 Listen: Communication Cost
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_communication_cost.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


💡 **Insight — Communication Cost of Ring Attention**

Let us think about the communication cost. In each round, every GPU sends one K chunk and one V chunk to its neighbor. With $P$ GPUs, chunk size $c = n/P$, and dimension $d_k$:

- **Per round:** each GPU sends $2 \times c \times d_k$ elements
- **Total rounds:** $P - 1$ (the first round uses local K, V so no communication is needed)
- **Total per GPU:** $2(P-1) \times c \times d_k = 2(P-1) \times \frac{n}{P} \times d_k$

As $P$ grows, this approaches $2n \cdot d_k$ — the same as an AllGather of all keys and values. But the ring pattern means we only ever need to hold **two chunks** in memory at once (current and incoming), rather than all $P$ chunks.

In [ ]:
# Let us compute these numbers for a realistic scenario
n_tokens = 131072  # 128K context
d_model = 128      # head dimension
num_heads = 32     # number of attention heads
bytes_per_param = 2  # fp16

for P in [2, 4, 8, 16]:
    chunk = n_tokens // P
    # Memory for attention scores: chunk x chunk per head
    attn_memory_gb = num_heads * chunk * chunk * bytes_per_param / 1e9
    # Memory for K,V per GPU: 2 chunks
    kv_memory_gb = 2 * num_heads * chunk * d_model * bytes_per_param / 1e9
    # Full attention matrix if no SP
    full_attn_gb = num_heads * n_tokens * n_tokens * bytes_per_param / 1e9

    print(f"P={P:2d} GPUs | chunk={chunk:6d} tokens | "
          f"attn block={attn_memory_gb:6.2f} GB | "
          f"K,V held={kv_memory_gb:.2f} GB | "
          f"(vs full attn: {full_attn_gb:.1f} GB)")

In [ ]:
#@title 🎧 Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/15_transition_to_moe.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Moe Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/16_moe_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


---

# Part B: Expert Parallelism

## 6. Building Intuition — Expert Parallelism

Imagine a high-end restaurant with eight specialist chefs: one for appetizers, one for sushi, one for curries, one for pasta, one for steaks, one for desserts, one for soups, and one for salads.

When an order comes in, a **host** (the router) looks at it and decides which chefs should handle it. A ramen order goes to the soup chef (70% contribution) and the appetizer chef for the side dish (30% contribution). A steak dinner might go to the steak chef (80%) and the salad chef (20%). The host does not send every order to every chef — that would be chaotic and wasteful.

This is exactly how a **Mixture of Experts (MoE)** works:
- **The chefs** are expert sub-networks (typically MLPs)
- **The host** is a learned router/gating network
- **The orders** are tokens flowing through the Transformer
- **The key insight**: only a few experts (top-K) are activated per token

This means a model can have an enormous total parameter count while using only a fraction for any given input. DeepSeek-V3 has 256 experts with 671B total parameters, but only the top-8 experts are active per token — just 37B active parameters. You get the capacity of a 671B model at the inference cost of a 37B model.

### Where Does Parallelism Come In?

With 256 experts, you cannot fit them all on one GPU. **Expert Parallelism** places different experts on different GPUs. The communication pattern is **All-to-All**: tokens must be dispatched from whatever GPU they are on to the GPU that holds their assigned expert, and then collected back.

In [ ]:
#@title 🎧 Listen: Moe Think
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/17_moe_think.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 🤔 Think About This

Before we dive into the math:
- If the router learns to send 90% of tokens to Expert 0 and 10% spread across the rest, what happens? (Hint: think about GPU utilization)
- Why use top-K (e.g., K=2) instead of sending each token to just one expert?
- If we have 8 experts on 8 GPUs, and batch size is 32 tokens, what is the maximum number of tokens any one GPU might have to process?

*These are real challenges that MoE systems must solve. We will address all of them.*

In [ ]:
# Quick status check
print("Section checkpoint ✓")

In [ ]:
#@title 🎧 Listen: Moe Math
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/18_moe_math.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. The Mathematics — Mixture of Experts

### The Router / Gating Network

Given a token representation $x \in \mathbb{R}^{d}$, the router computes gate weights over $N$ experts:

$$g(x) = \text{Softmax}(W_g \cdot x + b_g)$$

where $W_g \in \mathbb{R}^{N \times d}$ is the gating weight matrix. This gives us a probability distribution over experts: $g(x) \in \mathbb{R}^N$ with $\sum_i g_i(x) = 1$.

**Computationally:** We multiply the token's hidden state by a small linear layer and apply softmax. The output tells us how relevant each expert is for this particular token.

### Top-K Selection

We select only the $K$ experts with the highest gate values and renormalize:

$$\text{TopK}_i(x) = \begin{cases} \frac{g_i(x)}{\sum_{j \in \text{top-K}} g_j(x)} & \text{if } i \in \text{top-K indices} \\ 0 & \text{otherwise} \end{cases}$$

**Let us plug in some simple numbers.** Suppose we have $N=8$ experts with $K=2$ (top-2 routing). For a given token, the router outputs:

$$g(x) = [0.00, 0.05, 0.60, 0.00, 0.00, 0.35, 0.00, 0.00]$$

The top-2 experts are Expert 2 ($g_2 = 0.60$) and Expert 5 ($g_5 = 0.35$). After renormalization:

$$\hat{g}_2 = \frac{0.60}{0.60 + 0.35} = 0.632, \quad \hat{g}_5 = \frac{0.35}{0.60 + 0.35} = 0.368$$

### MoE Output

The final output is the weighted sum of the selected experts' outputs:

$$y = \sum_{i \in \text{top-K}} \hat{g}_i(x) \cdot E_i(x)$$

where $E_i(x)$ is the output of expert $i$. For our example:

$$y = 0.632 \cdot E_2(x) + 0.368 \cdot E_5(x)$$

**Computationally:** Only 2 of the 8 expert MLPs actually run a forward pass. The other 6 are skipped entirely. This is where the computational savings come from.

### Load Balancing Loss

Without encouragement, the router will often collapse — sending most tokens to just one or two experts while the rest sit idle. This is disastrous for Expert Parallelism because it means some GPUs are overloaded while others are idle.

The auxiliary load balancing loss encourages even distribution:

$$\mathcal{L}_{\text{aux}} = \alpha \cdot N \cdot \sum_{i=1}^{N} f_i \cdot P_i$$

where:
- $f_i = \frac{1}{T} \sum_{t=1}^{T} \mathbb{1}[\text{token } t \text{ is routed to expert } i]$ — the fraction of tokens assigned to expert $i$
- $P_i = \frac{1}{T} \sum_{t=1}^{T} g_i(x_t)$ — the average gate probability for expert $i$
- $\alpha$ is a small coefficient (typically 0.01)
- $N$ is the number of experts (scaling factor)

**What this says computationally:** If expert $i$ gets a disproportionate fraction of tokens ($f_i$ is high) AND the router gives it high probability ($P_i$ is high), the loss increases. The $N$ multiplier ensures the loss scale is independent of the number of experts.

**Numerical example:** With $N=4$ experts, $T=8$ tokens, $\alpha = 0.01$:

If tokens are perfectly balanced: $f_i = 0.25$ for all $i$, and $P_i \approx 0.25$:
$$\mathcal{L}_{\text{aux}} = 0.01 \times 4 \times 4 \times (0.25 \times 0.25) = 0.01$$

If tokens all go to Expert 0: $f_0 = 1.0, P_0 \approx 1.0$, others zero:
$$\mathcal{L}_{\text{aux}} = 0.01 \times 4 \times (1.0 \times 1.0) = 0.04$$

The collapsed case has 4x higher loss, pushing the router toward balance.

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/19_expert_mlp.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Let Us Build It — MoE from Scratch

### 8.1 Expert MLP

Each expert is simply a small two-layer MLP with ReLU activation. In a real model like DeepSeek-V3, each expert has billions of parameters. We will use small dimensions to keep everything fast.

In [ ]:
class ExpertMLP(nn.Module):
    """A single expert: 2-layer MLP with ReLU."""

    def __init__(self, d_model: int, d_ff: int):
        super().__init__()
        self.w1 = nn.Linear(d_model, d_ff)
        self.w2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """x: (..., d_model) -> (..., d_model)"""
        return self.w2(self.relu(self.w1(x)))

# Test a single expert
d_model, d_ff = 64, 128
expert = ExpertMLP(d_model, d_ff)
test_token = torch.randn(1, d_model)
output = expert(test_token)
print(f"Expert input:  {test_token.shape}")
print(f"Expert output: {output.shape}")
print(f"Expert parameters: {sum(p.numel() for p in expert.parameters()):,}")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/20_router.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 8.2 Router / Gating Network

The router is a simple linear layer followed by softmax. It maps each token's hidden state to a probability distribution over experts.

In [ ]:
class Router(nn.Module):
    """Gating network that routes tokens to experts."""

    def __init__(self, d_model: int, num_experts: int):
        super().__init__()
        self.gate = nn.Linear(d_model, num_experts, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (num_tokens, d_model)
        Returns:
            gate_probs: (num_tokens, num_experts) — softmax probs
        """
        logits = self.gate(x)
        return F.softmax(logits, dim=-1)

Let us test the router on some random tokens and visualize how it distributes gate weights across the 8 experts.

In [ ]:
# Test the router
num_experts = 8
router = Router(d_model, num_experts)

# Generate some test tokens
num_tokens = 16
tokens = torch.randn(num_tokens, d_model)
gate_probs = router(tokens)

print(f"Input:  {tokens.shape}")
print(f"Gates:  {gate_probs.shape}")
print(f"Sum of gates per token (should be 1.0): "
      f"{gate_probs.sum(dim=-1)[:4].tolist()}")
print(f"\nGate distribution for token 0:")
for i in range(num_experts):
    bar = '█' * int(gate_probs[0, i].item() * 40)
    print(f"  Expert {i}: {gate_probs[0, i].item():.3f} {bar}")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/21_viz_gate_distribution.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization 3: Gate Distribution for Sample Tokens

Let us see how the router distributes its attention across experts. The top-2 experts for each token are highlighted with red outlines.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle("Router Gate Weights for 4 Sample Tokens",
             fontsize=14, fontweight='bold')

colors = plt.cm.Set2(np.linspace(0, 1, num_experts))

for idx, ax in enumerate(axes.flat):
    probs = gate_probs[idx].detach().numpy()
    bars = ax.bar(range(num_experts), probs, color=colors,
                  edgecolor='black', linewidth=0.5)
    ax.set_title(f'Token {idx}', fontsize=11)
    ax.set_xlabel('Expert index')
    ax.set_ylabel('Gate weight')
    ax.set_ylim(0, max(probs) * 1.3)
    ax.set_xticks(range(num_experts))

    # Highlight top-2
    top2_indices = np.argsort(probs)[-2:]
    for i in top2_indices:
        bars[i].set_edgecolor('red')
        bars[i].set_linewidth(2.5)
        ax.text(i, probs[i] + 0.01, f'{probs[i]:.2f}',
                ha='center', fontsize=9, fontweight='bold', color='red')

plt.tight_layout()
plt.show()
print("Red-outlined bars = top-2 selected experts per token.")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/22_topk_gating.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 8.3 Top-K Selection and Routing

Now we implement the core routing logic: select the top-K experts for each token and renormalize the gate weights.

In [ ]:
def top_k_gating(gate_probs: torch.Tensor, k: int
                 ) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Select top-K experts per token and renormalize gate weights.

    Args:
        gate_probs: (num_tokens, num_experts) — softmax probs
        k: number of experts to select per token

    Returns:
        topk_weights: (num_tokens, k) — renormalized weights
        topk_indices: (num_tokens, k) — selected expert indices
    """
    # Get top-K gate values and their indices
    topk_weights, topk_indices = torch.topk(gate_probs, k, dim=-1)

    # Renormalize so weights sum to 1
    topk_weights = topk_weights / topk_weights.sum(dim=-1, keepdim=True)

    return topk_weights, topk_indices

Let us test the top-K gating and inspect which experts get selected for the first few tokens.

In [ ]:
# Test top-K gating
K = 2
weights, indices = top_k_gating(gate_probs, K)

print(f"Top-{K} gating results:")
print(f"  Weights shape: {weights.shape}")
print(f"  Indices shape: {indices.shape}")
print(f"\nFirst 4 tokens:")
for t in range(4):
    experts = indices[t].tolist()
    ws = weights[t].tolist()
    print(f"  Token {t}: Expert {experts[0]} (w={ws[0]:.3f}) + "
          f"Expert {experts[1]} (w={ws[1]:.3f})  [sum={sum(ws):.3f}]")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/23_viz_routing_matrix.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization 4: Token Routing Matrix

Let us visualize which tokens go to which experts — a routing matrix that shows the MoE dispatch pattern. Each cell shows the gate weight assigned to that token-expert pair.

In [ ]:
# Create a routing matrix: (num_tokens, num_experts)
routing_matrix = torch.zeros(num_tokens, num_experts)
for t in range(num_tokens):
    for j in range(K):
        expert_idx = indices[t, j].item()
        routing_matrix[t, expert_idx] = weights[t, j].item()

fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(routing_matrix.detach().numpy(), cmap='YlOrRd',
               aspect='auto')

ax.set_xlabel('Expert Index', fontsize=12)
ax.set_ylabel('Token Index', fontsize=12)
ax.set_title(f'Token Routing Matrix (Top-{K} Routing)',
             fontsize=14, fontweight='bold')
ax.set_xticks(range(num_experts))
ax.set_xticklabels([f'E{i}' for i in range(num_experts)])
ax.set_yticks(range(num_tokens))
ax.set_yticklabels([f'T{i}' for i in range(num_tokens)])

plt.colorbar(im, ax=ax, label='Gate weight')

We annotate the cells with their weight values and print the expert load statistics.

In [ ]:
# Annotate cells with weight values where non-zero
for t in range(num_tokens):
    for e in range(num_experts):
        val = routing_matrix[t, e].item()
        if val > 0.01:
            ax.text(e, t, f'{val:.2f}', ha='center', va='center',
                    fontsize=7,
                    color='white' if val > 0.4 else 'black')

plt.tight_layout()
plt.show()

# Expert load statistics
tokens_per_expert = (routing_matrix > 0).sum(dim=0).tolist()
print(f"\nTokens routed to each expert: {tokens_per_expert}")
print(f"Ideal (perfectly balanced): "
      f"{num_tokens * K / num_experts:.1f} tokens each")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/24_full_moe_layer.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 8.4 Full MoE Layer

Now let us assemble all the components into a complete Mixture of Experts layer. The MoE layer contains N expert MLPs and a router. For each input token, the router selects the top-K experts, dispatches the token through those experts, and combines their outputs with the renormalized gate weights.

In [ ]:
class MoELayer(nn.Module):
    """
    Mixture of Experts layer.

    Contains N expert MLPs and a router. Each token is dispatched
    to the top-K experts, and the output is the weighted sum.
    """

    def __init__(self, d_model: int, d_ff: int,
                 num_experts: int, top_k: int):
        super().__init__()
        self.num_experts = num_experts
        self.top_k = top_k
        self.router = Router(d_model, num_experts)
        self.experts = nn.ModuleList([
            ExpertMLP(d_model, d_ff) for _ in range(num_experts)
        ])

The forward pass: route tokens to experts, compute expert outputs, and combine with gate weights.

In [ ]:
    def forward(self, x: torch.Tensor
                ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Args:
            x: (num_tokens, d_model)
        Returns:
            output: (num_tokens, d_model)
            gate_probs: for load balancing loss
            topk_indices: for analysis
        """
        num_tokens, d_model = x.shape

        # Step 1: Route — get gate probabilities
        gate_probs = self.router(x)

        # Step 2: Select top-K experts
        topk_weights, topk_indices = top_k_gating(gate_probs, self.top_k)

        # Step 3: Dispatch and compute
        output = torch.zeros_like(x)
        for t in range(num_tokens):
            for j in range(self.top_k):
                expert_idx = topk_indices[t, j].item()
                weight = topk_weights[t, j]
                expert_output = self.experts[expert_idx](x[t:t+1])
                output[t] += weight * expert_output.squeeze(0)

        return output, gate_probs, topk_indices

As with ring attention, we need the class defined as a single unit for Python. Here is the complete `MoELayer`:

In [ ]:
class MoELayer(nn.Module):
    """Mixture of Experts layer with top-K routing."""

    def __init__(self, d_model, d_ff, num_experts, top_k):
        super().__init__()
        self.num_experts = num_experts
        self.top_k = top_k
        self.router = Router(d_model, num_experts)
        self.experts = nn.ModuleList([
            ExpertMLP(d_model, d_ff) for _ in range(num_experts)
        ])

    def forward(self, x):
        num_tokens, d_model = x.shape
        gate_probs = self.router(x)
        topk_weights, topk_indices = top_k_gating(
            gate_probs, self.top_k)
        output = torch.zeros_like(x)
        for t in range(num_tokens):
            for j in range(self.top_k):
                expert_idx = topk_indices[t, j].item()
                weight = topk_weights[t, j]
                expert_out = self.experts[expert_idx](x[t:t+1])
                output[t] += weight * expert_out.squeeze(0)
        return output, gate_probs, topk_indices

Let us test the full MoE layer and examine its parameter breakdown — total vs active parameters per token.

In [ ]:
# Test the full MoE layer
torch.manual_seed(42)
moe = MoELayer(d_model=64, d_ff=128, num_experts=8, top_k=2)

test_tokens = torch.randn(16, 64)
output, gate_probs, topk_indices = moe(test_tokens)

total_params = sum(p.numel() for p in moe.parameters())
expert_params = sum(p.numel() for p in moe.experts.parameters())
router_params = sum(p.numel() for p in moe.router.parameters())

print(f"MoE Layer Summary:")
print(f"  Input:           {test_tokens.shape}")
print(f"  Output:          {output.shape}")
print(f"  Total params:    {total_params:,}")
print(f"  Expert params:   {expert_params:,} "
      f"({expert_params/total_params*100:.1f}%)")
print(f"  Router params:   {router_params:,} "
      f"({router_params/total_params*100:.1f}%)")
print(f"  Params per expert: {expert_params // 8:,}")
print(f"  Active params per token: "
      f"{expert_params // 8 * 2 + router_params:,} "
      f"(2 experts + router)")
print(f"\n  Sparsity: {(1 - 2/8)*100:.0f}% of expert params "
      f"are unused per token")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/25_load_balancing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 8.5 Load Balancing Loss

Without the auxiliary loss, the router tends to collapse — sending most tokens to just one or two experts. Let us implement the loss and compare balanced vs collapsed routing.

In [ ]:
def load_balancing_loss(gate_probs: torch.Tensor,
                        topk_indices: torch.Tensor,
                        num_experts: int,
                        alpha: float = 0.01) -> torch.Tensor:
    """
    Compute the auxiliary load balancing loss.
    L_aux = alpha * N * sum_i(f_i * P_i)

    Args:
        gate_probs: (num_tokens, num_experts)
        topk_indices: (num_tokens, top_k)
        num_experts: N
        alpha: loss coefficient
    Returns:
        loss: scalar tensor
    """
    num_tokens = gate_probs.shape[0]

    # f_i: fraction of tokens routed to each expert
    expert_counts = torch.zeros(num_experts, device=gate_probs.device)
    for i in range(num_experts):
        expert_counts[i] = (topk_indices == i).any(dim=-1).float().sum()
    f = expert_counts / num_tokens

    # P_i: average gate probability for each expert
    P = gate_probs.mean(dim=0)

    # L_aux = alpha * N * sum(f_i * P_i)
    loss = alpha * num_experts * (f * P).sum()
    return loss

Now let us compute the loss for our current routing and compare it to the theoretical best (perfectly balanced) and worst (completely collapsed) cases.

In [ ]:
# Compute loss for our current routing
loss = load_balancing_loss(gate_probs, topk_indices, num_experts=8)
print(f"Load balancing loss: {loss.item():.6f}")

# What would it be if perfectly balanced?
perfect_f = torch.full((8,), 1/8)
perfect_P = torch.full((8,), 1/8)
perfect_loss = 0.01 * 8 * (perfect_f * perfect_P).sum().item()
print(f"Perfect balance loss: {perfect_loss:.6f}")

# What would it be if completely collapsed?
collapsed_f = torch.zeros(8)
collapsed_f[0] = 1.0
collapsed_P = torch.zeros(8)
collapsed_P[0] = 1.0
collapsed_loss = 0.01 * 8 * (collapsed_f * collapsed_P).sum().item()
print(f"Collapsed routing loss: {collapsed_loss:.6f}")
print(f"\nCollapsed is {collapsed_loss/perfect_loss:.0f}x worse than "
      f"balanced — the loss pushes toward balance.")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/26_viz_training_balancing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization 5: Training the Router with Load Balancing

Let us actually train a small MoE and watch the load balancing loss reshape the expert utilization over time. First, we define the training function that tracks expert usage at each step.

In [ ]:
def train_moe_with_balancing(num_steps: int = 300,
                             alpha: float = 0.01,
                             use_aux_loss: bool = True) -> dict:
    """Train a small MoE on random data and track expert usage."""
    torch.manual_seed(123)
    moe_train = MoELayer(d_model=32, d_ff=64,
                         num_experts=8, top_k=2)

    optimizer = torch.optim.Adam(moe_train.parameters(), lr=1e-3)
    target_fn = nn.Linear(32, 32, bias=False)
    target_fn.requires_grad_(False)

    history = {'step': [], 'task_loss': [], 'aux_loss': [],
               'expert_usage': [], 'max_load': [], 'min_load': []}

    for step in range(num_steps):
        x = torch.randn(64, 32)
        target = target_fn(x).detach()

        output, gate_probs, topk_idx = moe_train(x)

        task_loss = F.mse_loss(output, target)
        aux_loss = load_balancing_loss(
            gate_probs, topk_idx, 8, alpha=alpha)

        total_loss = task_loss + (aux_loss if use_aux_loss else 0.0)

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

The tracking portion: we record expert usage statistics every 10 steps to visualize the training dynamics.

In [ ]:
        # Track expert usage
        usage = torch.zeros(8)
        for i in range(8):
            usage[i] = (topk_idx == i).any(dim=-1).float().mean().item()

        if step % 10 == 0:
            history['step'].append(step)
            history['task_loss'].append(task_loss.item())
            history['aux_loss'].append(aux_loss.item())
            history['expert_usage'].append(usage.numpy().copy())
            history['max_load'].append(usage.max().item())
            history['min_load'].append(usage.min().item())

    return history

Now let us train the MoE both with and without the auxiliary load balancing loss, so we can see the difference.

In [ ]:
# Train with and without load balancing
print("Training WITH load balancing loss...")
hist_balanced = train_moe_with_balancing(
    num_steps=300, alpha=0.01, use_aux_loss=True)
print("Training WITHOUT load balancing loss...")
hist_unbalanced = train_moe_with_balancing(
    num_steps=300, alpha=0.01, use_aux_loss=False)
print("Done!")

We plot the expert usage curves over training. The top row shows per-expert utilization; the bottom row shows the gap between the most-used and least-used expert (load imbalance).

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle("Effect of Load Balancing Loss on Expert Utilization",
             fontsize=14, fontweight='bold')

# Top left: Expert usage over time (with balancing)
ax = axes[0, 0]
usage_data = np.array(hist_balanced['expert_usage'])
for i in range(8):
    ax.plot(hist_balanced['step'], usage_data[:, i],
            label=f'E{i}', alpha=0.7)
ax.set_title('Expert Usage — WITH Aux Loss', fontweight='bold')
ax.set_xlabel('Training step')
ax.set_ylabel('Fraction of tokens')
ax.axhline(y=2/8, color='black', linestyle='--', alpha=0.5,
           label='Ideal (0.25)')
ax.legend(fontsize=7, ncol=3)
ax.set_ylim(0, 0.7)

Top right panel: same plot but for training WITHOUT the auxiliary loss, to contrast the behavior.

In [ ]:
# Top right: Expert usage over time (without balancing)
ax = axes[0, 1]
usage_data_ub = np.array(hist_unbalanced['expert_usage'])
for i in range(8):
    ax.plot(hist_unbalanced['step'], usage_data_ub[:, i],
            label=f'E{i}', alpha=0.7)
ax.set_title('Expert Usage — WITHOUT Aux Loss', fontweight='bold')
ax.set_xlabel('Training step')
ax.set_ylabel('Fraction of tokens')
ax.axhline(y=2/8, color='black', linestyle='--', alpha=0.5,
           label='Ideal (0.25)')
ax.legend(fontsize=7, ncol=3)
ax.set_ylim(0, 0.7)

Bottom left panel: the gap between max and min expert load over training (WITH aux loss).

In [ ]:
# Bottom left: Max vs Min load (with balancing)
ax = axes[1, 0]
ax.fill_between(hist_balanced['step'], hist_balanced['min_load'],
                hist_balanced['max_load'], alpha=0.3, color='green',
                label='Load range')
ax.plot(hist_balanced['step'], hist_balanced['max_load'],
        color='green', label='Max load')
ax.plot(hist_balanced['step'], hist_balanced['min_load'],
        color='green', linestyle='--', label='Min load')
ax.set_title('Load Imbalance — WITH Aux Loss', fontweight='bold')
ax.set_xlabel('Training step')
ax.set_ylabel('Fraction of tokens')
ax.legend()

Bottom right panel: load imbalance WITHOUT the auxiliary loss, then finalize the figure.

In [ ]:
# Bottom right: Max vs Min load (without balancing)
ax = axes[1, 1]
ax.fill_between(hist_unbalanced['step'],
                hist_unbalanced['min_load'],
                hist_unbalanced['max_load'], alpha=0.3, color='red',
                label='Load range')
ax.plot(hist_unbalanced['step'], hist_unbalanced['max_load'],
        color='red', label='Max load')
ax.plot(hist_unbalanced['step'], hist_unbalanced['min_load'],
        color='red', linestyle='--', label='Min load')
ax.set_title('Load Imbalance — WITHOUT Aux Loss', fontweight='bold')
ax.set_xlabel('Training step')
ax.set_ylabel('Fraction of tokens')
ax.legend()

plt.tight_layout()
plt.show()

print("With load balancing: experts converge toward equal utilization.")
print("Without it: some experts dominate while others become "
      "'dead' experts.")

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/27_todo2_topk_manual.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. 🔧 Your Turn — Expert Parallelism

### TODO 2: Implement Top-K Gating from Scratch

Implement the `top_k_gating_manual` function without using `torch.topk`. This deepens your understanding of the selection and renormalization steps.

In [ ]:
def top_k_gating_manual(gate_logits: torch.Tensor, k: int
                        ) -> Tuple[torch.Tensor, torch.Tensor]:
    """Select top-K experts and renormalize weights.
    Do NOT use torch.topk — implement selection manually.
    Args:
        gate_logits: (num_tokens, num_experts) — raw logits
        k: number of experts to select per token
    Returns:
        topk_weights: (num_tokens, k) — renormalized weights
        topk_indices: (num_tokens, k) — expert indices
    """
    num_tokens, num_experts = gate_logits.shape

    # ============ TODO ============
    # Step 1: Softmax over experts
    gate_probs = ???  # YOUR CODE HERE

    # Step 2: Find top-k indices for each token
    # Hint: torch.argsort(tensor, dim=-1, descending=True)
    topk_indices = ???  # YOUR CODE HERE — (num_tokens, k)

    # Step 3: Gather the probabilities at those indices
    topk_weights = ???  # YOUR CODE HERE — (num_tokens, k)

    # Step 4: Renormalize
    topk_weights = ???  # YOUR CODE HERE
    # ==============================

    return topk_weights, topk_indices

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/28_todo2_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us verify your manual implementation against the reference `torch.topk` version.

In [ ]:
# ✅ Verification: Test your manual top-K gating

torch.manual_seed(42)
test_logits = torch.randn(8, 8)  # 8 tokens, 8 experts
k_test = 2

# Your implementation
your_weights, your_indices = top_k_gating_manual(test_logits, k_test)

# Reference implementation using torch.topk
ref_probs = F.softmax(test_logits, dim=-1)
ref_weights, ref_indices = torch.topk(ref_probs, k_test, dim=-1)
ref_weights = ref_weights / ref_weights.sum(dim=-1, keepdim=True)

print("Your results vs Reference:")
print(f"  Indices match: {torch.equal(your_indices, ref_indices)}")
print(f"  Weights close: "
      f"{torch.allclose(your_weights, ref_weights, atol=1e-5)}")
print(f"  Weights sum to 1: "
      f"{torch.allclose(your_weights.sum(dim=-1), torch.ones(8), atol=1e-5)}")

if (torch.equal(your_indices, ref_indices)
        and torch.allclose(your_weights, ref_weights, atol=1e-5)):
    print("\n✅ Your top-K gating implementation is correct!")
else:
    print("\n⚠️ Something is off. Check your steps.")
    print(f"  Your indices[:3]:\n  {your_indices[:3]}")
    print(f"  Ref indices[:3]:\n  {ref_indices[:3]}")

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/29_todo3_dispatch.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 3: Implement All-to-All Token Dispatch

In Expert Parallelism, each GPU holds one (or a few) experts. Tokens arrive at whichever GPU their data shard is on, but must be dispatched to the GPU that holds their assigned expert. This is the **All-to-All** communication pattern.

Implement the dispatch function below: given tokens and their expert assignments, reorganize so that each "GPU" receives exactly the tokens assigned to its expert.

In [ ]:
def all_to_all_dispatch(tokens: torch.Tensor,
                        expert_assignments: torch.Tensor,
                        num_experts: int
                        ) -> Tuple[List[torch.Tensor],
                                   List[torch.Tensor]]:
    """
    Dispatch tokens to assigned experts (simulating All-to-All).
    Args:
        tokens: (num_tokens, d_model)
        expert_assignments: (num_tokens,) — expert index
        num_experts: total number of experts
    Returns:
        dispatched_tokens: list[num_experts] of (n_i, d_model)
        original_indices: list[num_experts] of token indices
    """
    # ============ TODO ============
    dispatched_tokens = []
    original_indices = []

    for expert_id in range(num_experts):
        # Find tokens assigned to this expert
        mask = ???  # YOUR CODE HERE — boolean mask

        # Gather the tokens and their original indices
        ???  # YOUR CODE HERE

    # ==============================

    return dispatched_tokens, original_indices

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/30_todo3_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us test the dispatch with a known assignment pattern to verify each expert receives the correct tokens.

In [ ]:
# ✅ Verification: Test your All-to-All dispatch
torch.manual_seed(42)
test_tokens = torch.randn(12, 8)  # 12 tokens, d_model=8
test_assignments = torch.tensor(
    [0, 2, 1, 0, 3, 2, 1, 0, 3, 1, 2, 3])

dispatched, orig_idx = all_to_all_dispatch(
    test_tokens, test_assignments, num_experts=4)

print("Dispatch results:")
all_correct = True
expected = {0: [0, 3, 7], 1: [2, 6, 9],
            2: [1, 5, 10], 3: [4, 8, 11]}
for i in range(4):
    idx_list = orig_idx[i].tolist() if len(orig_idx[i]) > 0 else []
    print(f"  Expert/GPU {i}: {len(dispatched[i])} tokens, "
          f"indices = {idx_list}")
    if idx_list != expected[i]:
        all_correct = False
    for j, orig in enumerate(idx_list):
        if not torch.equal(dispatched[i][j], test_tokens[orig]):
            all_correct = False

Print the final verification result for the dispatch function.

In [ ]:
if all_correct:
    print("✅ Dispatch is correct! Each expert received the "
          "right tokens.")
    print("   In real EP, this would be an All-to-All collective "
          "across GPUs.")
else:
    print("⚠️ Check your dispatch logic.")

In [ ]:
#@title 🎧 Listen: Alltoall Cost
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/31_alltoall_cost.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


💡 **Insight — All-to-All Communication Cost**

The All-to-All operation is the most expensive part of Expert Parallelism. Let us understand its cost by computing the bytes transferred for a realistic configuration.

In [ ]:
# Communication analysis for All-to-All dispatch
batch_tokens = 4096          # tokens per micro-batch
d_model_real = 4096          # hidden dimension (e.g., LLaMA-7B)
num_experts_real = 64        # number of experts
top_k_real = 2               # top-K routing
bytes_per_param = 2          # fp16

tokens_dispatched = batch_tokens * top_k_real
bytes_per_transfer = d_model_real * bytes_per_param
total_comm_bytes = 2 * tokens_dispatched * bytes_per_transfer

print("All-to-All Communication Cost:")
print(f"  Batch tokens:    {batch_tokens:,}")
print(f"  Top-K:           {top_k_real}")
print(f"  Token transfers: {tokens_dispatched:,} (forward dispatch)")
print(f"  Bytes per token: {bytes_per_transfer:,} "
      f"({d_model_real} dims * {bytes_per_param} bytes)")
print(f"  Total comm:      {total_comm_bytes/1e6:.1f} MB per fwd pass")
print(f"  (x2 for forward+backward: "
      f"{2*total_comm_bytes/1e6:.1f} MB per step)")
print(f"\nFor comparison: an AllReduce of a "
      f"{d_model_real}x{d_model_real} matrix "
      f"= {d_model_real**2 * bytes_per_param / 1e6:.1f} MB")

In [ ]:
#@title 🎧 Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/32_transition_final.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


---

## 10. 🎯 Final Output — Putting It All Together

### 10.1 Ring Attention: Full Simulation

Let us run a complete Ring Attention simulation on a realistic sequence and verify correctness.

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/33_ring_full_sim.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Ring Attention: full simulation with a longer sequence
torch.manual_seed(42)
SEQ_LEN = 2048
D_K = 64
N_GPUS = 4
CHUNK = SEQ_LEN // N_GPUS

Q_sim = torch.randn(SEQ_LEN, D_K)
K_sim = torch.randn(SEQ_LEN, D_K)
V_sim = torch.randn(SEQ_LEN, D_K)

Q_ch, K_ch, V_ch = chunk_sequence(Q_sim, K_sim, V_sim, N_GPUS)

# Run both methods
output_std = standard_attention(Q_sim, K_sim, V_sim)
output_ring = ring_attention(Q_ch, K_ch, V_ch)

error = (output_ring - output_std).abs().max().item()
print(f"Ring Attention on {SEQ_LEN} tokens across {N_GPUS} GPUs")
print(f"  Chunk size: {CHUNK} tokens per GPU")
print(f"  Max error vs standard attention: {error:.2e}")
print(f"  ✅ Match!" if error < 1e-4 else "  ⚠️ Mismatch!")

Let us also analyze the memory savings Ring Attention provides at different sequence lengths and GPU counts.

In [ ]:
# Memory savings analysis
print("\n📊 Memory Analysis: Ring Attention vs Standard")
print("=" * 55)
for n in [2048, 8192, 32768, 131072, 524288]:
    full_mem = n * n * 2 / 1e9  # fp16 attention matrix
    for p in [4, 8]:
        c = n // p
        ring_mem = c * c * 2 / 1e9  # only one block at a time
        savings = (1 - ring_mem / full_mem) * 100
        if n == 2048 or n == 131072:
            print(f"  n={n:>7,} | P={p} GPUs | "
                  f"Full: {full_mem:>8.2f} GB | "
                  f"Ring block: {ring_mem:>8.4f} GB | "
                  f"Savings: {savings:.1f}%")
    if n == 2048 or n == 131072:
        print()

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/34_moe_final_demo.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 10.2 MoE: Full Working Layer with Routing Visualization

Let us build a final MoE layer and examine its parameter efficiency.

In [ ]:
# Build a realistic-ish MoE layer
torch.manual_seed(42)
D_MODEL_MOE = 128
D_FF_MOE = 256
N_EXPERTS = 8
TOP_K_MOE = 2
NUM_TOKENS_MOE = 64

moe_final = MoELayer(D_MODEL_MOE, D_FF_MOE, N_EXPERTS, TOP_K_MOE)

# Process a batch of tokens
input_tokens = torch.randn(NUM_TOKENS_MOE, D_MODEL_MOE)
output_tokens, final_gates, final_routing = moe_final(input_tokens)

print(f"MoE Layer — Final Demonstration")
print(f"  Experts:           {N_EXPERTS}")
print(f"  Top-K:             {TOP_K_MOE}")
print(f"  Input tokens:      {NUM_TOKENS_MOE}")
print(f"  Input shape:       {input_tokens.shape}")
print(f"  Output shape:      {output_tokens.shape}")

total_p = sum(p.numel() for p in moe_final.parameters())
active_p = (sum(p.numel() for p in moe_final.experts[0].parameters())
            * TOP_K_MOE
            + sum(p.numel() for p in moe_final.router.parameters()))
print(f"  Total parameters:  {total_p:,}")
print(f"  Active per token:  {active_p:,}")
print(f"  Sparsity:          {(1 - active_p/total_p)*100:.1f}%")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/35_viz_overview.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization 6: Complete Overview

Now let us create a 4-panel overview showing both Sequence and Expert Parallelism side by side. We start with the Ring Attention schematic and block computation schedule (top row).

In [ ]:
fig = plt.figure(figsize=(18, 12))
fig.suptitle("🎯 Sequence & Expert Parallelism — Complete Overview",
             fontsize=16, fontweight='bold', y=0.98)

# --- Panel 1: Ring Attention schematic (top left) ---
ax1 = fig.add_subplot(2, 2, 1)
ring_colors = ['#2196F3', '#4CAF50', '#FF9800', '#E91E63']
angles_final = [math.pi/2 - i * 2*math.pi/4 for i in range(4)]
positions_final = [(0.9*math.cos(a), 0.9*math.sin(a))
                   for a in angles_final]

# Draw ring
theta_ring = np.linspace(0, 2*np.pi, 100)
ax1.plot(0.9*np.cos(theta_ring), 0.9*np.sin(theta_ring),
         'gray', linewidth=1, alpha=0.3)

for i in range(4):
    x, y = positions_final[i]
    circle_patch = plt.Circle((x, y), 0.25,
                               color=ring_colors[i],
                               alpha=0.85, zorder=5)
    ax1.add_patch(circle_patch)
    ax1.text(x, y + 0.05, f'GPU {i}', ha='center', va='center',
             fontsize=9, fontweight='bold', color='white', zorder=6)
    ax1.text(x, y - 0.08, f'Q[{i*CHUNK}:{(i+1)*CHUNK}]',
             ha='center', va='center', fontsize=6,
             color='white', zorder=6)

Add curved arrows for K,V transfer direction and labels to Panel 1, then create Panel 2 showing the block computation schedule.

In [ ]:
# Draw curved arrows for Panel 1
for i in range(4):
    arc = mpatches.FancyArrowPatch(
        positions_final[i], positions_final[(i+1)%4],
        connectionstyle=f"arc3,rad=0.3",
        arrowstyle='->', color='gray', linewidth=1.5,
        mutation_scale=15, zorder=3)
    ax1.add_patch(arc)

ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)
ax1.set_aspect('equal')
ax1.set_title(f'Ring Attention — {SEQ_LEN} tokens, {N_GPUS} GPUs',
              fontweight='bold', fontsize=11)
ax1.text(0, -1.35, 'K,V rotate clockwise each round',
         ha='center', fontsize=9, style='italic', color='gray')
ax1.axis('off')

Panel 2 (top right): the block computation schedule showing which round each Q-K block pair is computed in.

In [ ]:
# --- Panel 2: Attention blocks (top right) ---
ax2 = fig.add_subplot(2, 2, 2)
block_img = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        block_img[i, j] = (i - j) % 4

im2 = ax2.imshow(block_img, cmap='Set2', aspect='equal')
ax2.set_xticks(range(4))
ax2.set_yticks(range(4))
ax2.set_xticklabels([f'K[{i}]' for i in range(4)])
ax2.set_yticklabels([f'Q[{i}]' for i in range(4)])
ax2.set_xlabel('Key chunks')
ax2.set_ylabel('Query chunks')
ax2.set_title('Block Computation Schedule',
              fontweight='bold', fontsize=11)
for i in range(4):
    for j in range(4):
        round_num = int(block_img[i, j])
        ax2.text(j, i, f'Round {round_num}', ha='center',
                 va='center', fontsize=8, fontweight='bold')
cbar2 = plt.colorbar(im2, ax=ax2, ticks=[0, 1, 2, 3])
cbar2.set_label('Round')

Now the bottom row: Panel 3 showing the MoE token routing heatmap, and Panel 4 showing expert utilization.

In [ ]:
# --- Panel 3: MoE routing (bottom left) ---
ax3 = fig.add_subplot(2, 2, 3)
routing_viz = torch.zeros(min(32, NUM_TOKENS_MOE), N_EXPERTS)
topk_w, topk_i = top_k_gating(
    final_gates[:32].detach(), TOP_K_MOE)
for t in range(min(32, NUM_TOKENS_MOE)):
    for j in range(TOP_K_MOE):
        routing_viz[t, topk_i[t, j]] = topk_w[t, j]

im3 = ax3.imshow(routing_viz.numpy(), cmap='YlOrRd', aspect='auto')
ax3.set_xlabel('Expert index')
ax3.set_ylabel('Token index')
ax3.set_title(f'MoE Token Routing (Top-{TOP_K_MOE})',
              fontweight='bold', fontsize=11)
ax3.set_xticks(range(N_EXPERTS))
ax3.set_xticklabels([f'E{i}' for i in range(N_EXPERTS)])
plt.colorbar(im3, ax=ax3, label='Gate weight')

Panel 4 (bottom right): expert utilization bar chart with the ideal utilization line.

In [ ]:
# --- Panel 4: Expert utilization (bottom right) ---
ax4 = fig.add_subplot(2, 2, 4)
expert_usage_final = torch.zeros(N_EXPERTS)
for i in range(N_EXPERTS):
    expert_usage_final[i] = (
        (final_routing == i).any(dim=-1).float().mean())

bars4 = ax4.bar(range(N_EXPERTS),
                expert_usage_final.detach().numpy(),
                color=[plt.cm.Set2(i/N_EXPERTS)
                       for i in range(N_EXPERTS)],
                edgecolor='black', linewidth=0.5)
ax4.axhline(y=TOP_K_MOE/N_EXPERTS, color='red', linestyle='--',
            linewidth=1.5,
            label=f'Ideal ({TOP_K_MOE/N_EXPERTS:.2f})')
ax4.set_xlabel('Expert index')
ax4.set_ylabel('Fraction of tokens')
ax4.set_title('Expert Utilization',
              fontweight='bold', fontsize=11)
ax4.set_xticks(range(N_EXPERTS))
ax4.set_xticklabels([f'E{i}' for i in range(N_EXPERTS)])
ax4.legend()

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Scale Numbers
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/36_scale_numbers.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 10.3 Scale Numbers: The Real World

Let us put these techniques in context with real production systems.

In [ ]:
print("=" * 65)
print("  How These Techniques Scale in Production")
print("=" * 65)
print()
print("┌──────────────────────────────────────────────────────────┐")
print("│  SEQUENCE PARALLELISM                                    │")
print("├──────────────────────────────────────────────────────────┤")
print("│  GPT-3 (2020):    2K context    → no SP needed           │")
print("│  LLaMA-2 (2023):  4K context    → Megatron-SP enough     │")
print("│  Claude 3 (2024): 200K context  → Ring Attention needed  │")
print("│  Gemini 1.5:      1M+ context   → Ring Attention + Flash │")
print("│                                                          │")
print("│  Memory: 128K tokens → 62 GB for attention matrix        │")
print("│  With 8-way Ring: 8 GB per GPU (8x reduction)            │")
print("└──────────────────────────────────────────────────────────┘")

Expert Parallelism in real systems, and the full 5D parallelism summary.

In [ ]:
print("┌──────────────────────────────────────────────────────────┐")
print("│  EXPERT PARALLELISM                                      │")
print("├──────────────────────────────────────────────────────────┤")
print("│  Switch Transformer: 128 experts, top-1,  1.6T total     │")
print("│  Mixtral 8x7B:       8 experts,  top-2,  47B total       │")
print("│  DeepSeek-V3:       256 experts, top-8, 671B total       │")
print("│                                                          │")
print("│  Key insight: 671B parameters at 37B inference cost       │")
print("│  That is a 18x parameter-to-compute multiplier!          │")
print("└──────────────────────────────────────────────────────────┘")
print()
print("🎉 You now understand all 5 dimensions of parallelism!")
print()
print("   1. Data Parallelism       — split the batch")
print("   2. Tensor Parallelism     — split the layers (columns/rows)")
print("   3. Pipeline Parallelism   — split the depth (stages)")
print("   4. Sequence Parallelism   — split the sequence (ring/SP)")
print("   5. Expert Parallelism     — split the experts (MoE)")
print()
print("   Modern systems like DeepSeek-V3 use ALL FIVE "
     "simultaneously.")

In [ ]:
#@title 🎧 Listen: Reflection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/37_reflection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


---

## 11. Reflection and Next Steps

### 🤔 Reflection Questions

1. **Why does Ring Attention need the online softmax trick?** Consider what would happen if each GPU computed `softmax(Q_local @ K_block.T / sqrt(d_k)) @ V_block` independently and then summed the results. Would the answer be correct? Why or why not?

2. **Top-1 vs Top-2 vs Top-8 routing:** With top-1, each token sees only one expert — faster but less expressive. With top-8 (like DeepSeek-V3), each token blends 8 expert opinions — more expressive but 8x more compute. What is the trade-off? Why did DeepSeek choose top-8?

3. **Load balancing failure modes:** Without the auxiliary loss, the router often collapses to using 1-2 experts. Why does this happen from an optimization perspective? (Hint: think about gradient flow — if an expert is never selected, does it ever get gradients?)

4. **Combining SP and EP:** In a real system, a single Transformer layer might use Tensor Parallelism for attention, Sequence Parallelism for LayerNorm, and Expert Parallelism for the MoE-MLP. How do the communication patterns interact?

### 🏆 Optional Challenges

1. **Causal Ring Attention:** Our implementation computes full (bidirectional) attention. Modify it to support causal masking, where tokens can only attend to earlier positions. What blocks in the attention matrix become zero?

2. **Expert Capacity Factor:** In practice, each expert has a maximum number of tokens it can process (the "capacity"). Implement a capacity factor of 1.25 — each expert can handle at most $1.25 \times T/N$ tokens. Tokens that exceed capacity are dropped. How does this interact with load balancing?

3. **Shared Expert (DeepSeek-V3):** DeepSeek-V3 uses one "shared" expert that processes ALL tokens, plus top-K from the remaining routed experts. Implement this: `y = E_shared(x) + sum(g_i * E_i(x))`. Why might this help?

4. **Flash Attention integration:** Our ring attention recomputes the full $c \times c$ attention block. In practice, each block would use Flash Attention (tiled, memory-efficient). Research how Flash Attention tiles the computation and sketch how it would integrate into the ring.

In [ ]:
#@title 🎧 Wrap-Up
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/38_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
print("🎉 Congratulations!")
print()
print("You have now built and understood:")
print("  ✅ Ring Attention — distributing long sequences "
     "across GPUs in a ring")
print("  ✅ Online softmax — combining partial attention "
     "results correctly")
print("  ✅ MoE routing — top-K expert selection "
     "with renormalized gates")
print("  ✅ Load balancing — auxiliary loss to prevent "
     "expert collapse")
print("  ✅ All-to-All dispatch — the communication pattern "
     "of Expert Parallelism")
print()
print("In the final notebook (Notebook 6), we will bring all "
     "5 dimensions")
print("together into a unified parallelism strategy — showing "
     "how real")
print("systems like Megatron-LM and DeepSeek-V3 combine "
     "DP + TP + PP + SP + EP")
print("to train models with hundreds of billions of parameters "
     "across")
print("thousands of GPUs.")